#soft voting
- randomforest
- xgbm
- lgbm
- weight 1,1,1




In [7]:
! pip install  scikit-learn pandas numpy matplotlib seaborn Optuna xgboost
# pycaret



In [11]:
! pip install --no-binary :all: lightgbm



In [12]:
! pip install lightgbm==3.3.2

  Using cached lightgbm-3.3.2.tar.gz (1.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached wheel-0.44.0-py3-none-any.whl.metadata (2.3 kB)
Using cached wheel-0.44.0-py3-none-any.whl (67 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for lightgbm (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [96 lines of output]
      INFO:root:running bdist_wheel
      INFO:root:running build
      INFO:root:running build_py
      INFO:root:creating build/lib/lightgbm
      INFO:root:copying lightgbm/callback.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/compat.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/plotting.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/__init__.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/engine.py -> build/lib/lightgbm
      INFO:root:copying lightgbm/dask.py -> build/lib/lightgbm
      IN

In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
import seaborn as sns


# data import
df=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/hy-0919-ml/bank.csv')
df

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/hy-0919-ml/bank.csv'

In [4]:


# 데이터 준비 (예시 데이터를 df라고 가정)
X = df.drop(['deposit', 'duration'], axis=1)
df['deposit'] = df['deposit'].replace({'yes': 1, 'no': 0})
y = df['deposit']


# 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 데이터 분할
# 데이터를 train+validation과 test로 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train+validation을 다시 train과 validation으로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)





# 모델 생성
rf = RandomForestClassifier(n_estimators=100, random_state=42)
# gbm = GradientBoostingClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(n_estimators=100, random_state=42)
lgbm = LGBMClassifier(n_estimators=100, random_state=42)

# Voting Classifier 생성
voting_clf = VotingClassifier([
    ('rf', rf),
    ('xgb', xgb),
    ('lgbm', lgbm)
], voting='soft', weights=[1, 1, 1])

# 모델 학습
voting_clf.fit(X_train, y_train)

# 예측
y_pred = voting_clf.predict(X_test)
y_pred_proba = voting_clf.predict_proba(X_test)[:, 1]

# AUC와 Recall 계산
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred, pos_label=1)

print(f"AUC: {auc:.4f}")
print(f"Recall: {recall:.4f}")


# ## 특성 중요도 (Random Forest 기준)
# importances = voting_clf.named_estimators_['rf'].feature_importances_
# indices = np.argsort(importances)[::-1]
# feature_names = X.columns

# plt.figure(figsize=(10, 6))
# plt.title("Feature Importances")
# plt.bar(range(X.shape[1]), importances[indices])
# plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=90)
# plt.tight_layout()
# plt.show()

AUC: 0.8003
Recall: 0.6289


##gridsearch 사용


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score, make_scorer
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from lightgbm import early_stopping


# 데이터 준비
X = df.drop(['deposit', 'duration'], axis=1)
df['deposit'] = df['deposit'].replace({'yes': 1, 'no': 0})
y = df['deposit']

# 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 데이터를 train+validation과 test로 분할
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# train+validation을 다시 train과 validation으로 분할
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.2, random_state=42)

# 커스텀 스코어 함수 (AUC와 Recall의 조화 평균)
def custom_score(y_true, y_pred_proba):
    auc = roc_auc_score(y_true, y_pred_proba)
    recall = recall_score(y_true, y_pred_proba > 0.5, pos_label=1)
    return 2 * (auc * recall) / (auc + recall)

custom_scorer = make_scorer(custom_score, needs_proba=True)

# 각 모델에 대한 파라미터 그리드 정의
param_grids = {
    'rf': {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5]
    },
    'xgb': {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7]
    },
    'lgbm': {
        'learning_rate': [0.01, 0.1],
        'num_leaves': [31, 63]
    }
}
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

best_models = {}

for name, model in [('rf', RandomForestClassifier(random_state=42)),
                    ('xgb', XGBClassifier(random_state=42)),
                    ('lgbm', LGBMClassifier(random_state=42))]:
    print(f"Performing GridSearch for {name}...")

    if name == 'lgbm':
        param_grids[name]['n_estimators'] = [1000]  # Set a high number of estimators
        # Use GridSearchCV for parameter tuning
        grid_search = GridSearchCV(model, param_grids[name], cv=3, scoring=custom_scorer, n_jobs=-1)
        grid_search.fit(X_train, y_train)

        # Extract the best model and perform early stopping separately
        best_lgbm_model = grid_search.best_estimator_
        best_lgbm_model.fit(X_train, y_train,
                            eval_set=[(X_val, y_val)],
                            eval_metric='auc',
                            early_stopping_rounds=50,
                            verbose=False)

        best_models[name] = best_lgbm_model
    else:
        grid_search = GridSearchCV(model, param_grids[name], cv=3, scoring=custom_scorer, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_models[name] = grid_search.best_estimator_

    print(f"Best parameters for {name}: {grid_search.best_params_}")
    print(f"Best score for {name}: {grid_search.best_score_:.4f}")





# Voting Classifier 생성
voting_clf = VotingClassifier([
    ('rf', best_models['rf']),
    ('xgb', best_models['xgb']),
    ('lgbm', best_models['lgbm'])
], voting='soft', weights=[1, 1, 1])

# 모델 학습 (train+validation 데이터 사용)
voting_clf.fit(X_train_val, y_train_val)

# 예측
y_pred = voting_clf.predict(X_test)
y_pred_proba = voting_clf.predict_proba(X_test)[:, 1]

# AUC와 Recall 계산
auc = roc_auc_score(y_test, y_pred_proba)
recall = recall_score(y_test, y_pred, pos_label=1)

print(f"Final AUC: {auc:.4f}")
print(f"Final Recall: {recall:.4f}")

ModuleNotFoundError: No module named 'xgboost'

## optuna 사용

In [ ]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import roc_auc_score, recall_score
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt

# 데이터 준비
X = df.drop(['deposit', 'duration'], axis=1)
df['deposit'] = df['deposit'].replace({'yes': 1, 'no': 0})
y = df['deposit']

# 범주형 변수 원-핫 인코딩
X = pd.get_dummies(X, drop_first=True)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

def objective(trial):
    # RandomForest 파라미터
    rf_params = {
        'n_estimators': trial.suggest_int('rf_n_estimators', 100, 300),
        'max_depth': trial.suggest_int('rf_max_depth', 5, 30),
        'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 10)
    }

    # XGBoost 파라미터
    xgb_params = {
        'n_estimators': trial.suggest_int('xgb_n_estimators', 100, 300),
        'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 0.1),
        'max_depth': trial.suggest_int('xgb_max_depth', 3, 10)
    }

    # LightGBM 파라미터
    lgbm_params = {
        'n_estimators': trial.suggest_int('lgbm_n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 1e-3, 0.1),
        'num_leaves': trial.suggest_int('lgbm_num_leaves', 31, 127)
    }

    # 모델 생성
    rf = RandomForestClassifier(**rf_params, random_state=42)
    xgb = XGBClassifier(**xgb_params, random_state=42)
    lgbm = LGBMClassifier(**lgbm_params, random_state=42)

    # Voting Classifier 생성
    voting_clf = VotingClassifier([
        ('rf', rf),
        ('xgb', xgb),
        ('lgbm', lgbm)
    ], voting='soft', weights=[1, 1, 1])

    # 모델 학습
    voting_clf.fit(X_train, y_train)

    # 예측
    y_pred = voting_clf.predict(X_test)
    y_pred_proba = voting_clf.predict_proba(X_test)[:, 1]

    # AUC와 Recall 계산
    auc = roc_auc_score(y_test, y_pred_proba)
    recall = recall_score(y_test, y_pred)

    return auc, recall

# Optuna 스터디 생성 및 최적화 실행
study = optuna.create_study(
    study_name="bank_marketing_voting_clf",
    storage="sqlite:///optuna_results.db",
    directions=["maximize", "maximize"],
    load_if_exists=True
)

study.optimize(objective, n_trials=50)

# 최적의 파라미터 출력
best_trial = study.best_trials[0]  # Pareto optimal solutions의 첫 번째 trial
print("Best trial:")
print("  AUC: ", best_trial.values[0])
print("  Recall: ", best_trial.values[1])
print("  Params: ")
for key, value in best_trial.params.items():
    print("    {}: {}".format(key, value))

# 최적의 파라미터로 모델 재학습
best_params = best_trial.params
rf = RandomForestClassifier(
    n_estimators=best_params['rf_n_estimators'],
    max_depth=best_params['rf_max_depth'],
    min_samples_split=best_params['rf_min_samples_split'],
    random_state=42
)
xgb = XGBClassifier(
    n_estimators=best_params['xgb_n_estimators'],
    learning_rate=best_params['xgb_learning_rate'],
    max_depth=best_params['xgb_max_depth'],
    random_state=42
)
lgbm = LGBMClassifier(
    n_estimators=best_params['lgbm_n_estimators'],
    learning_rate=best_params['lgbm_learning_rate'],
    num_leaves=best_params['lgbm_num_leaves'],
    random_state=42
)

best_voting_clf = VotingClassifier([
    ('rf', rf),
    ('xgb', xgb),
    ('lgbm', lgbm)
], voting='soft', weights=[1, 1, 1])

best_voting_clf.fit(X_train, y_train)

# 최종 성능 평가
y_pred = best_voting_clf.predict(X_test)
y_pred_proba = best_voting_clf.predict_proba(X_test)[:, 1]

final_auc = roc_auc_score(y_test, y_pred_proba)
final_recall = recall_score(y_test, y_pred)

print(f"Final Soft Voting Results:")
print(f"AUC: {final_auc:.4f}")
print(f"Recall: {final_recall:.4f}")

# 결과 시각화
def plot_optimization_history(study):
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

def plot_param_importances(study):
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

def plot_pareto_front(study):
    fig = optuna.visualization.plot_pareto_front(study)
    fig.show()

plot_optimization_history(study)
plot_param_importances(study)
plot_pareto_front(study)

[I 2024-09-19 16:27:59,417] Using an existing study with name 'bank_marketing_voting_clf' instead of creating a new one.
<ipython-input-12-0edf83b10153>:33: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('xgb_learning_rate', 1e-3, 0.1),
<ipython-input-12-0edf83b10153>:40: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate': trial.suggest_loguniform('lgbm_learning_rate', 1e-3, 0.1),
[I 2024-09-19 16:28:12,907] Trial 84 finished with values: [0.8028939284089502, 0.6204311152764761] and parameters: {'rf_n_estimators': 248, 'rf_max_depth': 5, 'rf_min_samples_split': 8, 'xgb_n_estimators': 133, 'xgb_learning_rate': 0.0931044

# pycaret 안됨 issue -> 이후부터 pycaret (lgmg)





#pycaret , AutoML

lightgbm	Light Gradient Boosting Machine	0.7369	0.7930	0.7369	0.7424	0.7335	0.4675	0.4753	1.1510

gbc	Gradient Boosting Classifier	0.7345	0.7914	0.7345	0.7416	0.7302	0.4618	0.4717	2.0840



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

df=pd.read_csv('/content/bank.csv')
df

In [ ]:
best_model = create_model(best_models[0])
tuned_model = tune_model(best_model, optimize='AUC')


evaluate_model(tuned_model)
plot_model(tuned_model, plot='auc')
plot_model(tuned_model, plot='confusion_matrix')
plot_model(tuned_model, plot='feature_importance')

In [ ]:
# 필요한 라이브러리 설치
!pip install pycaret pandas matplotlib seaborn

In [ ]:


# 데이터 처리 및 모델 훈련 코드
import pandas as pd
from pycaret.classification import *

# 데이터 로드
data = df

# 데이터프레임으로 변환
df = pd.DataFrame(data)

# PyCaret 설정
# duration 변수를 제거한 상태로 모델링 진행
clf = setup(data=df,
            target='deposit',
            ignore_features=['duration'],  # duration 변수를 feature selection에서 제외
            session_id=123,
            # silent=True,
            use_gpu=False)

# 상위 3개의 모델 훈련 및 비교
top3_models = compare_models(n_select=3)

# 상위 모델들의 성능 확인
print(top3_models)

# # 각 모델별 시각화
# plot_model(top3_models[0], plot='auc')  # 첫 번째 모델의 AUC 곡선 시각화
# plot_model(top3_models[1], plot='confusion_matrix')  # 두 번째 모델의 혼동 행렬 시각화
# plot_model(top3_models[2], plot='feature')  # 세 번째 모델의 feature 중요도 시각화

# 모델 블렌딩 (앙상블)
blended_model = blend_models(estimator_list=top3_models)

# 블렌딩된 모델 평가
evaluate_model(blended_model)


# optuna 사용

basemodel: Light Gradient Boosting Machine

In [ ]:
! pip install optuna

In [ ]:
import optuna
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from lightgbm import early_stopping, log_evaluation

# 데이터 로드 및 전처리 (이전 코드와 동일)
df = pd.read_csv('/content/bank.csv')

def prepare_data(data, target):
    le = LabelEncoder()
    data[target] = le.fit_transform(data[target])
    return data, le

df, label_encoder = prepare_data(df, 'deposit')

X = df.drop(['deposit', 'duration'], axis=1)
y = df['deposit']

for col in X.select_dtypes(include=['object']):
    X[col] = LabelEncoder().fit_transform(X[col])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Optuna 최적화 함수 (이전과 동일)
def objective(trial):
    params = {
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': trial.suggest_int('num_leaves', 20, 3000),
        'max_depth': trial.suggest_int('max_depth', 3, 12),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 1.0),
        'n_estimators': trial.suggest_int('n_estimators', 50, 10000),
        'min_child_samples': trial.suggest_int('min_child_samples', 1, 300),
        'subsample': trial.suggest_uniform('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'random_state': 42
    }

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = []

    for train_idx, val_idx in cv.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]

        train_data = lgb.Dataset(X_train_fold, label=y_train_fold)
        val_data = lgb.Dataset(X_val_fold, label=y_val_fold)

        model = lgb.train(
            params,
            train_data,
            valid_sets=[val_data],
            num_boost_round=params['n_estimators'],
            callbacks=[early_stopping(100), log_evaluation(period=0)]
        )

        cv_scores.append(model.best_score['valid_0']['auc'])

    return np.mean(cv_scores)

# SQLite 데이터베이스를 사용하여 Optuna 연구 생성
study_name = "lightgbm_optimization"
storage_name = "sqlite:///optuna_studies.db"
study = optuna.create_study(study_name=study_name, storage=storage_name, direction='maximize', load_if_exists=True)

# 최적화 실행
study.optimize(objective, n_trials=50, n_jobs=-1, timeout=3600)

# 최적의 하이퍼파라미터 출력
print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

# 최적화된 모델 생성 및 훈련
best_params = study.best_params
best_params['objective'] = 'binary'
best_params['metric'] = 'auc'

train_data = lgb.Dataset(X_train, label=y_train)
final_model = lgb.train(best_params, train_data, num_boost_round=best_params['n_estimators'])

# 테스트 세트에 대한 예측 및 성능 평가
y_pred = final_model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

accuracy = accuracy_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
print(f"AUC: {auc:.4f}")

# 특성 중요도 출력
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': final_model.feature_importance()
}).sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(feature_importance.head(10))

# 레이블 디코딩 함수
def decode_predictions(predictions):
    return label_encoder.inverse_transform(predictions)

# 디코딩된 예측 결과 예시
decoded_predictions = decode_predictions(y_pred_binary)
print("\nSample of decoded predictions:", decoded_predictions[:10])

# 모든 시도의 결과 확인
print("\nAll trials:")
trial_data = study.trials_dataframe()
print(trial_data.sort_values('value', ascending=False).head())

# 가장 좋은 시도의 하이퍼파라미터 저장
best_params_df = pd.DataFrame([study.best_params])
best_params_df.to_csv('best_lightgbm_params.csv', index=False)
print("\nBest parameters saved to 'best_lightgbm_params.csv'")

# featur selection + xgboost

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel

# 데이터 로드
df = pd.read_csv('/content/bank.csv')

# 전처리
le = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = le.fit_transform(df[column])

# 특성과 타겟 분리
X = df.drop(['deposit', 'duration'], axis=1)
y = df['deposit']

# 훈련 및 테스트 세트 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 초기 XGBoost 모델 생성
initial_model = XGBClassifier(random_state=42)
initial_model.fit(X_train, y_train)

# 특성 중요도 기반 특성 선택
selector = SelectFromModel(initial_model, prefit=True, threshold='median')
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

# 선택된 특성으로 새 XGBoost 모델 훈련
model = XGBClassifier(random_state=42)

# 하이퍼파라미터 그리드 정의
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.3],
    'n_estimators': [100, 200, 300],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0]
}

# GridSearchCV를 사용한 하이퍼파라미터 튜닝
grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                           cv=3, scoring='recall', n_jobs=-1)
grid_search.fit(X_train_selected, y_train)

# 최적의 모델 선택
best_model = grid_search.best_estimator_

# 테스트 세트에 대한 예측
y_pred = best_model.predict(X_test_selected)

# 정확도와 재현율 계산
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred, pos_label=1)

print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")

# 특성 중요도 출력
feature_importance = best_model.feature_importances_
feature_names = X.columns[selector.get_support()]
for name, importance in zip(feature_names, feature_importance):
    print(f"{name}: {importance}")